<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-bert-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install datasets
!pip install evaluate
import os
if not os.path.exists('/content/drive/MyDrive/lora-bert-tiny/'):
    os.makedirs('/content/drive/MyDrive/lora-bert-tiny/')
!pip freeze > /content/drive/MyDrive/lora-bert-tiny/requirements.txt

In [3]:
import torch
import torch.nn as nn
import numpy as np
from transformers import set_seed
set_seed(0)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
num_runs = 5

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [8]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

# Full Fine-Tuning



In [9]:
from transformers import AutoModelForSequenceClassification

ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
import time

start_time = time.time()

ft_results = []
iterations = len(batch_sizes)*len(learning_rates)*num_runs
progress = 0
for run in range(1, num_runs+1):
  for size in batch_sizes:
    for rate in learning_rates:
      progress += 1
      print(f"Progress: {progress}/{iterations}")
      ft_result = {"run":run, "batch_size": size, "learning_rate": rate}

      ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

      training_args = TrainingArguments(
          "ft-trainer",
          eval_strategy="epoch",
          per_device_eval_batch_size=size,
          per_device_train_batch_size=size,
          num_train_epochs=epochs,
          learning_rate=rate,
          disable_tqdm=True,
          report_to="none"
      )

      trainer = Trainer(
          ft_model,
          training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["validation"],
          data_collator=data_collator,
          processing_class=tokenizer,
          compute_metrics=compute_metrics
      )

      ft_result = ft_result | trainer.train().metrics | trainer.evaluate()
      ft_results.append(ft_result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_minutes = runtime_seconds / 60

ft_max = ft_results[0]
for i, result in enumerate(ft_results):
  if result['eval_f1'] > ft_max['eval_f1']:
    ft_max = result

print(f'\n================ \
        \nRuntime: {runtime_minutes} minutes \
        \nBest Result: \
        \nbatch_size={ft_max["batch_size"]}, \
        \nlearning_rate={ft_max["learning_rate"]}, \
        \neval_f1={ft_max["eval_f1"]}')

import csv

with open('/content/drive/MyDrive/lora-bert-tiny/ft-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(ft_results[0].keys()))

    writer.writeheader()
    for result in ft_results:
      writer.writerow(result)

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Progress: 1/100


{'eval_loss': 0.5835773944854736, 'eval_accuracy': 0.7181372549019608, 'eval_f1': 0.8275862068965517, 'eval_runtime': 0.662, 'eval_samples_per_second': 616.338, 'eval_steps_per_second': 77.042, 'epoch': 1.0}
{'loss': 0.5911, 'grad_norm': 3.7462449073791504, 'learning_rate': 0.00021830065359477123, 'epoch': 1.0893246187363834}


KeyboardInterrupt: 

# Low-Rank Adaptation (LoRA)

In [ ]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [ ]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

In [ ]:
def configure_lora_model(model, rank, alpha):
  for i, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False
  params, trainable_params = get_trainable_parameters(model)
  print(f"Total Parameters={params}, Trainable Parameters={trainable_params}")

  return (params, trainable_params)

In [ ]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

In [ ]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [ ]:
from transformers import Trainer, TrainingArguments
import time

start_time = time.time()

results = []
iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)*num_runs
progress = 0
for run in range(1, num_runs+1):
  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          progress += 1
          print(f"Progress: {progress}/{iterations}")
          result = {"run": run, "rank": rank, "alpha": alpha, "batch_size": size, "learning_rate": rate}

          params, trainable_params = configure_lora_model(lora_model, rank, alpha)
          result["parameters"] = params
          result["trainable_parameters"] = trainable_params

          training_args = TrainingArguments(
              "lora-trainer",
              eval_strategy="epoch",
              per_device_eval_batch_size=size,
              per_device_train_batch_size=size,
              num_train_epochs=epochs,
              learning_rate=rate,
              disable_tqdm=True,
              report_to="none"
          )

          trainer = Trainer(
              lora_model,
              training_args,
              train_dataset=tokenized_datasets["train"],
              eval_dataset=tokenized_datasets["validation"],
              data_collator=data_collator,
              processing_class=tokenizer,
              compute_metrics=compute_metrics
          )

          result = result | trainer.train().metrics | trainer.evaluate()
          results.append(result)

end_time = time.time()

runtime_seconds = end_time - start_time
runtime_hours = runtime_seconds / 3600

max = results[0]
for i, result in enumerate(results):
  if result['eval_f1'] > max['eval_f1']:
    max = result

print(f'\n================ \
        \nRuntime: {runtime_hours} hours \
        \nBest Result: \
        \nrank={max["rank"]}, \
        \nalpha={max["alpha"]}, \
        \nparameters={max["parameters"]}, \
        \ntrainable_parameters={max["trainable_parameters"]}, \
        \nbatch_size={max["batch_size"]}, \
        \nlearning_rate={max["learning_rate"]}, \
        \neval_f1={max["eval_f1"]}')

import csv

with open('/content/drive/MyDrive/lora-bert-tiny/lora-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))

    writer.writeheader()
    for result in results:
      writer.writerow(result)